# Mosquito Research

In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('poster')

pd.options.display.max_columns = None

In [2]:
def get_lags(data, x, lag):
    lag_avg = pd.DataFrame(data[x])
    
    def assign_na(x):
        try:
            y = np.float64(x)
        except:
            y = np.NaN
        return y

    lag_avg[x] = lag_avg[x].apply(assign_na)

    for i in range(1, lag):
        lag_avg['lag_{}'.format(i)] = lag_avg[x].copy().shift(i)

    ten_day_avg = []
    for i, row in lag_avg.iterrows():
        ten_day_avg.append(row.mean(skipna=True))

    return pd.Series(ten_day_avg)


In [3]:
! ls ../data

WNV_Pop_Age.csv        distance-mapping.ipynb test copy.csv
WNV_Pop_Age_Temp.csv   model_data.csv         test.csv
census_age.csv         sampleSubmission.csv   train.csv
census_info.csv        spray.csv              weather.csv
data-clean.ipynb       spray_traps.csv


Mosquito research [paper](https://academic.oup.com/ee/article-abstract/44/4/1022/2465753?redirectedFrom=fulltext)

In [4]:
train = pd.read_csv('../data/train.csv')

In [5]:
species_dummies = pd.get_dummies(train['Species'])

In [6]:
species_dummies.head(1)

,CULEX ERRATICUS,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS
0,0,0,1,0,0,0,0


In [7]:
df = train.join(species_dummies)

In [8]:
df.rename(columns={c: c.lower().replace(' ', '_') for c in df.columns}, inplace=True)

In [9]:
df['date'] = pd.to_datetime(df['date'])

In [10]:
df['year'] = df['date'].dt.year

In [11]:
df['week'] = df['date'].apply(lambda x: x.isocalendar()[1])

In [12]:
df.shape

(10506, 21)

In [13]:
df.head(1)

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,culex_erraticus,culex_pipiens,culex_pipiens/restuans,culex_restuans,culex_salinarius,culex_tarsalis,culex_territans,year,week
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0,0,1,0,0,0,0,2007,22


In [14]:
spray_traps = pd.read_csv('../data/spray_traps.csv')
spray_traps['Date'] = pd.to_datetime(spray_traps['Date'])
spray_traps['year'] = spray_traps['Date'].dt.year
spray_traps['week'] = spray_traps['Date'].apply(lambda x: x.isocalendar()[1])


In [15]:
summed = spray_traps.drop('spray_distance', axis=1).groupby(['trap', 'year', 'week']).sum().reset_index()
meaned = spray_traps.drop('spray', axis=1).groupby(['trap', 'year', 'week']).mean().reset_index()
spray_traps = summed.merge(meaned)

In [16]:
spray_traps.head()

,trap,year,week,spray,spray_distance
0,T001,2013,30,274,0.008212
1,T002,2013,33,326,0.010247
2,T004,2013,30,38,0.010787
3,T005,2013,33,279,0.008101
4,T006,2011,36,925,0.006450


In [17]:
df = df.merge(spray_traps, on=['year', 'week', 'trap'], how='left')

In [18]:
df.shape

(10506, 23)

In [19]:
df['spray'] = df['spray'].replace(np.NaN, 0.0)
df['spray_distance'] = df['spray_distance'].replace(np.NaN, 0.0)

In [20]:
df.head()

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,culex_erraticus,culex_pipiens,culex_pipiens/restuans,culex_restuans,culex_salinarius,culex_tarsalis,culex_territans,year,week,spray,spray_distance
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,0,1,0,0,0,0,2007,22,0.0,0.0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,0,0,1,0,0,0,2007,22,0.0,0.0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,0,0,0,1,0,0,0,2007,22,0.0,0.0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,0,0,1,0,0,0,0,2007,22,0.0,0.0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,0,0,0,1,0,0,0,2007,22,0.0,0.0


In [21]:
weather = pd.read_csv('../data/weather.csv')

In [22]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [23]:
weather.Date = pd.to_datetime(weather.Date)
weather['year'] = weather['Date'].dt.year
weather['week'] = weather['Date'].apply(lambda x: x.isocalendar()[1])

In [24]:
df_2 = pd.DataFrame()

df_2['ten_day_avg_percip'] = get_lags(weather, 'PrecipTotal', 10)
df_2['ten_day_avg_temp'] = get_lags(weather, 'Tavg', 10)
df_2['ten_day_avg_dewpoint'] = get_lags(weather, 'DewPoint', 10)
df_2['ten_day_avg_pressure'] = get_lags(weather, 'StnPressure', 10)
df_2['ten_day_avg_windspeed'] = get_lags(weather, 'AvgSpeed', 10)


In [25]:
df_2.head()


,ten_day_avg_percip,ten_day_avg_temp,ten_day_avg_dewpoint,ten_day_avg_pressure,ten_day_avg_windspeed
0,0.0,67.0,51.0,29.100,9.200000
1,0.0,67.5,51.0,29.140,9.400000
2,0.0,62.0,48.0,29.220,10.733333
3,0.0,59.5,46.5,29.275,11.400000
4,0.0,58.8,45.2,29.298,11.500000


In [26]:
weather_dates = pd.DataFrame(weather.Date)


In [27]:
w = weather_dates.join(df_2)


In [28]:
w = w.groupby('Date').mean().reset_index()


In [29]:
w = w.rename(columns={c: c.lower() for c in w.columns})


In [30]:
model_data = df.merge(w, on="date").set_index('date')


In [31]:
model_data.head(1)


,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,culex_erraticus,culex_pipiens,culex_pipiens/restuans,culex_restuans,culex_salinarius,culex_tarsalis,culex_territans,year,week,spray,spray_distance,ten_day_avg_percip,ten_day_avg_temp,ten_day_avg_dewpoint,ten_day_avg_pressure,ten_day_avg_windspeed
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0,0,1,0,0,0,0,2007,22,0.0,0.0,0.200893,65.6,50.6,29.4375,7.9


In [32]:
print(model_data.columns)


Index(['address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'nummosquitos', 'wnvpresent', 'culex_erraticus', 'culex_pipiens',
       'culex_pipiens/restuans', 'culex_restuans', 'culex_salinarius',
       'culex_tarsalis', 'culex_territans', 'year', 'week', 'spray',
       'spray_distance', 'ten_day_avg_percip', 'ten_day_avg_temp',
       'ten_day_avg_dewpoint', 'ten_day_avg_pressure',
       'ten_day_avg_windspeed'],
      dtype='object')


In [33]:
model_data = model_data.drop(['address', 'species', 'block', 'street', 'trap', 'year', 'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy'], axis=1)


In [34]:
model_data.head()

,nummosquitos,wnvpresent,culex_erraticus,culex_pipiens,culex_pipiens/restuans,culex_restuans,culex_salinarius,culex_tarsalis,culex_territans,week,spray,spray_distance,ten_day_avg_percip,ten_day_avg_temp,ten_day_avg_dewpoint,ten_day_avg_pressure,ten_day_avg_windspeed
date,,,,,,,,,,,,,,,,,
2007-05-29,1,0,0,0,1,0,0,0,0,22,0.0,0.0,0.200893,65.6,50.6,29.4375,7.9
2007-05-29,1,0,0,0,0,1,0,0,0,22,0.0,0.0,0.200893,65.6,50.6,29.4375,7.9
2007-05-29,1,0,0,0,0,1,0,0,0,22,0.0,0.0,0.200893,65.6,50.6,29.4375,7.9
2007-05-29,1,0,0,0,1,0,0,0,0,22,0.0,0.0,0.200893,65.6,50.6,29.4375,7.9
2007-05-29,4,0,0,0,0,1,0,0,0,22,0.0,0.0,0.200893,65.6,50.6,29.4375,7.9


In [35]:
model_data.to_csv('../data/model_data.csv')
